In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sc

In [2]:
class Jackknife:
    def __init__( self, len_data, binsize ):
        self.binsize = binsize
        self.nbins = int(len_data/self.binsize)
        self.N = self.binsize * self.nbins
        self.jack_avg = []
        self.est = 0
        self.var_est = 0

    def set( self, func, list_of_data ):
        for i in range( self.nbins ):
            self.jack_avg.append( func( i, self.binsize, list_of_data ) )

    def do_it( self ):
        for i in range( 0, self.nbins ):
            self.est += self.jack_avg[i]
        self.est /= self.nbins

        for i in range( 0, self.nbins ):
            self.var_est += ( self.jack_avg[i] - self.est )**2
        self.var_est /= self.nbins
        self.var_est *= self.nbins -1

    def mean( self ):
        return self.est

    def var( self ):
        return self.var_est

    def err( self ):
        return np.sqrt(self.var_est)

def simple_mean(i, binsize, np_data):
    resmpld = np.delete(np_data, np.s_[i*binsize:(i+1)*binsize])
    return np.mean(resmpld)

def format_print(cen, err):
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'

def format_print_w_exact(exact, cen, err):
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'+' exact:'+'{ex:.{width}f}'.format(ex=exact, width=i+2)+' ['+'{num:.{width}f}'.format(num=abs(exact-cen)/err, width=2)+' sigma]'

In [3]:
def get_rhot(data0):
    # data0=topo_ori.transpose()[20]
    data=data0 # -np.mean(data0)
    rolled=np.array([np.roll(data,t) for t in range(int(np.size(data)/2))])
    ct=np.mean(data*rolled,axis=1)
    rhot=ct/ct[0]
    return rhot

def get_delta_rhot(data, rhot):
    Lambda=min(int(np.size(rhot)/2),100)
    delta_rhot_sq_=[]
    for t in np.arange(1,int(np.size(rhot)/4)):
        tmp=np.sum(np.array([ rhot[k+t]+rhot[k-t]-2.0*rhot[k]*rhot[t] for k in np.arange(1,t+Lambda+1) ])**2)
        delta_rhot_sq_.append(tmp/np.size(data))
    delta_rhot=np.sqrt( np.array(delta_rhot_sq_) )
    return delta_rhot

def get_tauint(data,rhot,wlis):
    tauint_list=np.array([ np.sum(rhot[1:w]) + 0.5 for w in wlis ])
    delta_tauint_sq=(4.0*wlis+2.0)/np.size(data) * tauint_list**2
    delta_tauint=np.sqrt(delta_tauint_sq)
    return tauint_list, delta_tauint

In [4]:
def Z(beta_, hext_):
    res = 0.0
    res += 2.0*np.cosh(4.0*hext_)*np.exp(12.0*beta_)
    res += 8.0*np.cosh(2.0*hext_)
    res += 6.0*np.exp(-4.0*beta_)
    return res

In [6]:
def mag_per_site(beta_, hext_):
    denom = 0.0
    denom += np.cosh(4.0*hext_)*np.exp(12.0*beta_)
    denom += 4.0*np.cosh(2.0*hext_)
    denom += 3.0*np.exp(-4.0*beta_)
    
    numer = 0.0
    numer += np.sinh(4.0*hext_)*np.exp(12.0*beta_)
    numer += 2.0*np.sinh(2.0*hext_)
    return numer/denom

In [59]:
nin=2000
nfin=10000
nint=1
nconf=(nfin-nin)/nint

In [60]:
beta=0.2
hext=0.1
lx=2
ly=2
print("beta="+str(beta)+", hext="+str(hext)
     +", lx="+str(lx)+", ly="+str(ly))
vlat=lx*ly

beta=0.2, hext=0.1, lx=2, ly=2


In [61]:
## CHECK

delta=1.0e-2

Z0=Z(beta, hext)
ZP=Z(beta, hext+0.5*delta)
ZM=Z(beta, hext-0.5*delta)

numeric=0.25*(ZP-ZM)/delta/Z0
exact=mag_per_site(beta, hext)

format_print_w_exact(exact, numeric, delta**2)

'0.284275(100) exact:0.2842570 [0.18 sigma]'

In [62]:
data=np.loadtxt("data")

In [63]:
binsize=10

In [64]:
jk = Jackknife(nconf, binsize)
jk.set(simple_mean, data.T[0])
jk.do_it()
mu_w0=jk.mean()
sig_w0=jk.err()
print(format_print_w_exact(mag_per_site(beta,hext), mu_w0, sig_w0))

0.288(19) exact:0.2843 [0.19 sigma]


In [65]:
mag_per_site(beta,hext)

0.28425698441839403

In [66]:
sig_w0

0.019161070597549373

In [17]:
mu_w0

0.09989267676767674

In [50]:
data=np.loadtxt("beta001h01.dat")

In [51]:
binsize=10

In [52]:
jk = Jackknife(nconf, binsize)
jk.set(simple_mean, data.T[0])
jk.do_it()
mu_w0=jk.mean()
sig_w0=jk.err()
print(format_print_w_exact(mag_per_site(beta,hext), mu_w0, sig_w0))

0.102(16) exact:0.1058 [0.24 sigma]
